In [1]:
import configparser
import gc
import logging
import pathlib as path
import sys
from collections import defaultdict
from itertools import chain
from collections import defaultdict
import configparser
import pathlib as path
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from idao.data_module import IDAODataModule
from idao.model import SimpleConv
import numpy as np
import torch
import os
import pathlib as path
from PIL import Image
from torchvision.datasets import DatasetFolder
from torch.utils.data import Dataset
import pathlib as path

import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl
import torch
from torch import nn
from torch.nn import functional as F

import configparser
import pathlib as path

import pytorch_lightning as pl
from pytorch_lightning import seed_everything

import configparser
import gc
import logging
import pathlib as path
import sys
from collections import defaultdict
from itertools import chain

import scikitplot as skplt
import torch
from more_itertools import bucket

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

#from idao.data_module import IDAODataModule
#from idao.model import SimpleConv



In [2]:
class IDAODataset(DatasetFolder):
    def name_to_energy(self, name):
        names = os.path.split(name)[-1].split("_")
        idx = [i for i, v in enumerate(names) if v == "keV"][0]
        return torch.tensor(float(names[idx - 1]))

    def name_to_index(self, name):
        return os.path.split(name)[-1].split('.')[0]

    def __getitem__(self, index: int):
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target, self.name_to_energy(path), self.name_to_index(path)

class InferenceDataset(Dataset):
    def __init__(self, main_dir, transform, loader=None):
        self.img_loaderj= img_loader
        self.main_dir = path.Path(main_dir)
        self.transform = transform
        self.all_imgs = list(self.main_dir.glob("*.png"))
        self.loader = loader

    def __len__(self):
        return len(self.all_imgs)

    def __getitem__(self, idx):
        img_loc = self.all_imgs[idx]
        image = self.loader(img_loc)
        tensor_image = self.transform(image)
        return tensor_image, img_loc.name

def img_loader(path: str):
    with Image.open(path) as img:
        img = np.array(img)
    return img


In [3]:
class IDAODataModule(pl.LightningDataModule):
    def __init__(self, data_dir: path.Path, batch_size: int, cfg):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.cfg = cfg

    def prepare_data(self):
        # called only on 1 GPU
        self.dataset = IDAODataset(
            root=self.data_dir.joinpath("train"),
            loader=img_loader,
            transform=transforms.Compose(
                [transforms.ToTensor(), transforms.CenterCrop(120)]  #CenterCrop(120)
            ),
            # TODO(kazeevn) use idiomatic torch
            target_transform=transforms.Compose(
                [
                    lambda num: (
                        torch.tensor([0, 1]) if num == 0 else torch.tensor([1, 0])
                    )
                ]
            ),
            extensions=self.cfg["DATA"]["Extension"],
        )

        self.test = InferenceDataset(
                    main_dir=self.data_dir.joinpath("test"),
                    loader=img_loader,
                    transform=transforms.Compose(
                        [transforms.ToTensor(), transforms.CenterCrop(120)]#CenterCrop(120)
                    ),
                )


    def setup(self, stage=None):
        # called on every GPU
        self.train, self.val = random_split(
            self.dataset, [10000, 3404], generator=torch.Generator().manual_seed(666)
        )

    def train_dataloader(self):
        return DataLoader(self.train, self.batch_size, shuffle=True, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.val, 1, num_workers=1, shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(
            self.test,
            self.batch_size,
            num_workers=0,
            shuffle=False
            )



In [4]:
class Print(nn.Module):
    """Debugging only"""

    def forward(self, x):
        print(x.size())
        return x


class Clamp(nn.Module):
    """Clamp energy output"""

    def forward(self, x):
        x = torch.clamp(x, min=0, max=30)
        return x


class SimpleConv(pl.LightningModule):
    def __init__(self, mode: ["classification", "regression"] = "classification"):
        super().__init__()
        self.mode = mode
        self.layer1 = nn.Sequential(
                    nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
                    nn.BatchNorm2d(16),
                    nn.ReLU(),
                    nn.MaxPool2d(6),
                    nn.Conv2d(16,32,4,stride=1,padding=1),
                    nn.BatchNorm2d(32),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=6,stride=3),
                    nn.Flatten(),
                )
        

        self.drop_out = nn.Dropout()

        self.fc1 = nn.Linear(800, 100)
        self.fc2 = nn.Linear(100, 2)  # for classification
        self.fc3 = nn.Linear(100, 1)  # for regression


        self.stem = nn.Sequential(
            self.layer1, self.drop_out, self.fc1,
            )
        if self.mode == "classification":
            self.classification = nn.Sequential(self.stem, self.fc2)
        else:
            self.regression = nn.Sequential(self.stem, self.fc3)

        self.train_acc = pl.metrics.Accuracy()
        self.valid_acc = pl.metrics.Accuracy()
        self.test_acc = pl.metrics.Accuracy()

    def training_step(self, batch, batch_idx):
        # --------------------------
        x_target, class_target, reg_target, _ = batch
        if self.mode == "classification":
            class_pred = self.classification(x_target.float())
            class_loss = F.binary_cross_entropy_with_logits(
                class_pred, class_target.float()
            )
            self.train_acc(torch.sigmoid(class_pred), class_target)
            self.log("train_acc", self.train_acc, on_step=True, on_epoch=False)
            self.log("classification_loss", class_loss)

            return class_loss

        else:
            reg_pred = self.regression(x_target.float())
            #             reg_loss = F.l1_loss(reg_pred, reg_target.float().view(-1, 1))
            reg_loss = F.mse_loss(reg_pred, reg_target.float().view(-1, 1))

            #             reg_loss = torch.sum(torch.abs(reg_pred - reg_target.float().view(-1, 1)) / reg_target.float().view(-1, 1))
            self.log("regression_loss", reg_loss)
            return reg_loss

    def training_epoch_end(self, outs):
        # log epoch metric
        if self.mode == "classification":
            self.log("train_acc_epoch", self.train_acc.compute())
        else:
            pass

    def validation_step(self, batch, batch_idx):
        x_target, class_target, reg_target, _ = batch
        if self.mode == "classification":
            class_pred = self.classification(x_target.float())
            class_loss = F.binary_cross_entropy_with_logits(
                class_pred, class_target.float()
            )
            self.valid_acc(torch.sigmoid(class_pred), class_target)
            self.log("valid_acc", self.valid_acc.compute())
            self.log("classification_loss", class_loss)
            return class_loss

        else:
            reg_pred = self.regression(x_target.float())
            #             reg_loss = F.l1_loss(reg_pred, reg_target.float().view(-1, 1))
            reg_loss = F.mse_loss(reg_pred, reg_target.float().view(-1, 1))

            #             reg_loss = torch.sum(torch.abs(reg_pred - reg_target.float().view(-1, 1)) / reg_target.float().view(-1, 1))
            self.log("regression_loss", reg_loss)
            return reg_loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def forward(self, x):
        if self.mode == "classification":
            class_pred = self.classification(x.float())
            return {"class": torch.sigmoid(class_pred)}
        else:
            reg_pred = self.regression(x.float())
            return {"energy": reg_pred}


In [5]:
def compute_predictions(mode, dataloader, checkpoint_path, cfg):
    torch.multiprocessing.set_sharing_strategy("file_system")
    logging.info("Loading checkpoint")
    model = SimpleConv.load_from_checkpoint(checkpoint_path, mode=mode)
    model = model.cpu().eval()

    dict_pred = defaultdict(list)
    if mode == "classification":
        logging.info("Classification model loaded")
    else:
        logging.info("Regression model loaded")

    for img, name in iter(dataloader):
        if mode == "classification":
            dict_pred["id"].extend(map(lambda x: x.strip('.png'), name))
            output = model(img)["class"].detach()[:, 1].numpy()
            dict_pred["particle"].extend(output)
        else:
            output = model(img)["energy"].detach().squeeze(1).numpy()
            dict_pred["energy"].extend(output)
            
    return dict_pred


In [6]:
def main():
    config = configparser.ConfigParser()
    config.read("./config.ini")
    PATH = path.Path(config["DATA"]["DatasetPath"])

    dataset_dm = IDAODataModule(
        data_dir=PATH, batch_size=512, cfg=config
    )

    dataset_dm.prepare_data()
    print(dataset_dm.dataset.class_to_idx)
    dataset_dm.setup()
    dl = dataset_dm.test_dataloader()

    dict_pred = defaultdict(list)
    for mode in ["regression", "classification"]:
        if mode == "classification":
            model_path = config["REPORT"]["ClassificationCheckpoint"]
        else:
            model_path = config["REPORT"]["RegressionCheckpoint"]

        dict_pred.update(compute_predictions(mode, dl, model_path, cfg=config))

    data_frame = pd.DataFrame(dict_pred,
                              columns=["id", "energy", "particle"])
    data_frame.set_index("id", inplace=True)
    data_frame.to_csv('submission_classification_T36.csv.gz',
                      index=True, header=True, index_label="id", columns=["particle"])
    data_frame.to_csv('submission_regression_T36.csv.gz',
                      index=True, header=True, index_label="id", columns=["energy"])


In [7]:
if __name__ == "__main__":
    main()

{'ER': 0, 'NR': 1}


/usr/local/lib/python3.6/dist-packages/deprecate/deprecation.py:115: LightningDeprecationWarning:

The `Accuracy` was deprecated since v1.3.0 in favor of `torchmetrics.classification.accuracy.Accuracy`. It will be removed in v1.5.0.



/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:718: UserWarning:

Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)

